In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import Window, functions as F, types as T

In [2]:
os.environ['PYSPARK_PYTHON']= sys.executable
spark = (
    SparkSession.builder
    .master("local[1]")
    .config("spark.executor.cores", "2")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

24/09/23 12:10:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
schema = T.StructType([
    T.StructField('camp_id', T.StringType()),
    T.StructField('control_group_flg', T.ShortType()),
    T.StructField('revenue', T.DoubleType()),
    T.StructField('margin', T.DoubleType()),
    T.StructField('conversion', T.ShortType()),
    T.StructField('user_id', T.StringType()),
])

statistics_df = spark.createDataFrame(pd.read_csv('data_sample.csv'))
statistics_df.show(5)

[Stage 0:>                                                          (0 + 1) / 1]

+-------+-----------------+-------+------+----------+--------------------+
|camp_id|control_group_flg|revenue|margin|conversion|             user_id|
+-------+-----------------+-------+------+----------+--------------------+
|ED_3755|                0|   4.77|   5.3|         1|6f9be979ec9f4aba9...|
|ED_3755|                0|   7.53|  8.12|         1|6a0a4bdd10e670717...|
|ED_3755|                0|  12.33|  5.24|         1|2b96123b22c810b53...|
|ED_3755|                0|   5.44|  4.32|         1|06d37e5b5d6c45a9a...|
|ED_3755|                0|    0.0|   0.0|         0|86fcb416dd0d2b9e7...|
+-------+-----------------+-------+------+----------+--------------------+
only showing top 5 rows



In [4]:
# Список ключевых столбцов групп
KEY_COLS = ['camp_id', 'control_group_flg']
# Список метрик, по которым считаем статистику
STATISTICS_COLS = ['revenue', 'margin', 'conversion']
# Количество бутстрап операций
BS_ITERS = 10000
ALPHA = 0.05

In [5]:
# udf для создания массива количества появлений строки в каждой итерации
pois_udf = F.udf(lambda size: np.random.poisson(1, size).tolist(), 
                 T.ArrayType(T.IntegerType()))

result_df = (
    statistics_df
    .withColumn('iter_cnt', F.lit(BS_ITERS))
    # создаем массив с количеством вхождений строки в каждую итерацию
    .withColumn('poisson_array', pois_udf(F.col('iter_cnt')))
    # делаем posexplode, сохраняя порядковый номер итерации и количество вхождений
    .select(
        KEY_COLS 
        + [F.posexplode('poisson_array').alias('iter_num', 'poisson')] 
        + STATISTICS_COLS
    )
    # Убираем лишние строки, которые не участвуют в итерации бутстрапа
    .filter(F.col('poisson') != 0)
    # Группируем по ключевым полям и номеру итерации
    .groupBy(KEY_COLS  + ['iter_num'])
    # Считаем сумму метрик и количество клиентов в каждой итерации
    .agg(*(
        [F.sum(F.col('poisson') * F.col(stat_col)).alias(stat_col) 
         for stat_col in STATISTICS_COLS] 
        + 
        [F.sum(F.col('poisson')).alias('total_cnt')]
    ))
)

# Рассчитываем значение средних для каждой итерации
for stat_col in STATISTICS_COLS:
    result_df = (result_df
                 .withColumn(stat_col, F.col(stat_col) / F.col('total_cnt')))

In [6]:
# Сохраняем результат и приводим к единому виду с остальными тетрадками
result = result_df.toPandas()
# Минусуем метрики контрольной группы, чтобы сложить можно было для получения разницы
result.loc[result['control_group_flg'] == 1, STATISTICS_COLS] *= -1
# Можно собирать и сразу в pyspark
result = (
    result
    .groupby(['camp_id', 'iter_num'], as_index=False).sum()
    .groupby('camp_id', as_index=False)[STATISTICS_COLS].quantile([ALPHA/2, 1-ALPHA/2])
    .groupby('camp_id', as_index=False).agg(list)
)

result

,camp_id,revenue,margin,conversion
0,ED_3755,"[0.16715380990486672, 1.2306713983908648]","[-0.8067694136025351, 0.2941336974675275]","[0.011820438005005308, 0.06960119682338302]"


In [7]:
spark.stop()